# Data Transormation

## Install/Import packages & define key varribles and functions

In [19]:
# Run install script
# %chmod +x setup_jupyterlab.sh
# %./setup_jupyterlab.sh

# Import necessary libraries for the script to function.
import pandas as pd
import csv, json, re, os, venn, json
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
import matplotlib.patches as mpatches

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings

from sklearn.decomposition import PCA

from functools import partial
import seaborn as sns
from scipy.stats import pearsonr
from itertools import combinations

from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


In [4]:
merged_df = pd.read_csv('Merged_Dataframe_20241216_104635.csv')
# Automatically load group_data.json from the specified directory
file_path = os.path.join( 'group_data3.json')
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        group_data = json.load(json_file)
    print(f"Loaded group data from {file_path}")
    print(f"{len(group_data)} groups imported from file")

else:
    print(f"File {file_path} does not exist. Please check the path.")
    print(f"Importing group data from defined in earlier in the current session ")
    from my_functions_datatransformation import group_data
group_data = {key: group_data[key] for key in list(group_data.keys())[:42]}


Loaded group data from group_data3.json
47 groups imported from file


### Find sums and Counts

In [15]:
def sum_of_count_and_summed_abs(merged_df, group_data):
    # Initialize lists to store results
    results = []
    
    # Process each group in group_data
    for group_id, group_info in group_data.items():
        grouping_variable = group_info['grouping_variable']
        abundance_columns = group_info['abundance_columns']
        
        # Calculate totals for all columns in this group
        total_abundance = 0
        total_count = 0
        
        for col in abundance_columns:
            # Convert column to numeric, treating non-convertible values as NaN
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
            
            # Add to totals
            total_count += merged_df[col].count()
            total_abundance += merged_df[col].sum(skipna=True)
        
        # Calculate averages
        num_columns = len(abundance_columns)
        avg_count = total_count / num_columns if num_columns > 0 else 0
        avg_abundance = total_abundance / num_columns if num_columns > 0 else 0
        
        results.append({
            'Base_Sample_ID': grouping_variable,
            'Summed_Abundance': avg_abundance,
            'Sum_of_Count': avg_count
        })
    
    # Convert to DataFrame
    final_df = pd.DataFrame(results)
    
    return final_df

In [17]:
### Generate Summary table for all samples
summary_df = sum_of_count_and_summed_abs(merged_df,group_data)
summary_df

,Base_Sample_ID,Summed_Abundance,Sum_of_Count
0,Control,2.131632e+10,1070.888889
1,New_Formula,1.928497e+10,993.747126
2,Human_Milk,1.474354e+10,944.250000
3,Human_Milk_Intestinal,3.007412e+10,603.500000
4,Human_Milk_Gastric,1.339986e+10,1504.750000
5,Human_Milk_Feed,7.566503e+08,724.500000
6,Control_Feed,4.541658e+08,842.666667
7,Control_Gastric,7.544717e+09,1563.000000
8,Control_Intestinal,5.595007e+10,807.000000
9,New_Formula_Feed,3.316648e+09,800.344828


In [44]:
def bioactive_function_count_and_abundance_sum_avg(df, group_data):
    # Dictionaries to store results
    summed_function_count = {}
    unique_function_counts = {}
    unique_function_count_averages = {}
    unique_function_absorbance = {}
    summed_function_abundance = {}

    # Define separate DataFrames based on 'Marked as' values
    both_df = df[df['Marked as'] == 'Bovine;Human']
    hum_df = df[df['Marked as'] == 'Human']
    bov_df = df[df['Marked as'] == 'Bovine']
    
    # Concatenate hum_df with both_df and bov_df with both_df
    hum_combined_df = pd.concat([hum_df, both_df])
    bov_combined_df = pd.concat([bov_df, both_df])



    # Iterate over each group in the dictionary
    for group_key, group_info in group_data.items():
        grouping_variable = group_info['grouping_variable']
        abundance_columns = f'Avg_Abs_{grouping_variable}'
        # Determine the subset of the DataFrame based on 'Marked as' values
        # Check if both groups specifically start with "Human_Milk_"
        if grouping_variable.startswith("Human_"):
            df = hum_combined_df
        else:
            df = bov_combined_df       
        # Ensure abundance_columns is treated as a list
        if isinstance(abundance_columns, str):
            abundance_columns = [abundance_columns]
            
        # Ensure the 'unique ID' and 'function' columns are included in the filtered DataFrame
        relevant_columns = ['unique ID', 'function'] + abundance_columns
        
        # Filter the DataFrame for the relevant columns and drop any rows with NaN values in these columns
        temp_filter_df = df.loc[:, relevant_columns]
        temp_filter_df = temp_filter_df[(temp_filter_df[abundance_columns] != 0).all(axis=1) & temp_filter_df[abundance_columns].notna().all(axis=1)]

        # Drop duplicates based on 'unique ID', keeping the first occurrence
        filtered_df = temp_filter_df.drop_duplicates(subset='unique ID')

        # Calculate the number of unique peptides based on the 'unique ID' column
        unique_peptide_count = filtered_df['unique ID'].nunique()
        
        # Calculate the Summed all abundance values before splitting by function
        total_sum = filtered_df[abundance_columns].sum().sum()

        # Track the total summed abundance for the current group
        summed_function_abundance[grouping_variable] = total_sum

        # Track the unique peptide counts
        summed_function_count[grouping_variable] = unique_peptide_count

        # Split 'function' column into individual functions using .loc
        filtered_df.loc[:, 'function'] = filtered_df['function'].str.split(';')

        # Explode the 'function' column to handle each function separately
        exploded_df = filtered_df.explode('function')

        # Strip white spaces around the functions using .loc
        exploded_df.loc[:, 'function'] = exploded_df['function'].str.strip()

        # Count the occurrences of each unique function
        function_counts = exploded_df['function'].value_counts().to_dict()

        # Track the function counts for the current group
        unique_function_counts[grouping_variable] = function_counts

        # Calculate the average occurrences of each function per sample
        num_columns_in_group = len(abundance_columns)
        function_averages = {func: count / num_columns_in_group for func, count in function_counts.items()}
        unique_function_count_averages[grouping_variable] = function_averages

        # Group by 'function' and calculate the mean for each abundance column
        function_grouped = exploded_df.groupby('function')[abundance_columns].sum()

        # Calculate the Summed the average absorbance values for each function
        function_sum = function_grouped.sum(axis=1)

        # Track the Summed average absorbance values by function for the current group
        unique_function_absorbance[grouping_variable] = function_sum.to_dict()
    return (
        summed_function_count,
        unique_function_counts,
        unique_function_count_averages,
        unique_function_absorbance,
        summed_function_abundance
    )

# Example usage with debugging
summed_function_count, unique_function_counts, unique_function_count_averages, unique_function_absorbance, summed_function_absorbance = bioactive_function_count_and_abundance_sum_avg(merged_df, group_data)


In [49]:
# Create a DataFrame for unique peptide counts
peptide_count_df= pd.DataFrame.from_dict(summed_function_count, orient='index', columns=['Counts of peptides'])

# Create a DataFrame for unique function counts
function_count_df= pd.DataFrame.from_dict(unique_function_counts, orient='index').fillna(0).astype(int)

# Concatenate the DataFrames
combined_count_df = pd.concat([peptide_count_df, function_count_df], axis=1).T
combined_count_df

,Control,New_Formula,Human_Milk,Human_Milk_Intestinal,Human_Milk_Gastric,Human_Milk_Feed,Control_Feed,Control_Gastric,Control_Intestinal,New_Formula_Feed,...,G_Intestinal,H_Feed,H_Gastric,H_Intestinal,I_Feed,I_Gastric,I_Intestinal,J_Feed,J_Gastric,J_Intestinal
Counts of peptides,1916,2184,1794,549,1432,768,902,1461,829,1644,...,761,815,1443,728,757,1383,729,709,1466,764
ACE-inhibitory,173,187,20,17,14,7,78,130,98,144,...,91,66,128,88,65,131,87,64,139,88
Antimicrobial,168,192,52,10,50,27,78,125,82,159,...,71,67,124,73,63,124,74,66,127,78
Antioxidant,123,129,1,1,1,1,60,98,70,107,...,65,56,97,65,51,92,61,53,101,57
Immunomodulatory,53,54,11,0,11,10,28,41,26,44,...,23,21,38,23,20,40,24,20,42,27
DPP-IV Inhibitory,41,71,1,1,0,0,9,21,35,64,...,35,11,23,38,11,21,33,9,25,32
Anticancer,20,25,2,2,1,1,11,12,9,21,...,8,11,12,8,8,10,9,11,14,10
Antithrombotic,20,21,0,0,0,0,15,17,6,18,...,4,13,17,4,12,15,4,15,18,6
Osteoanabolic,16,18,0,0,0,0,15,14,6,18,...,6,14,15,5,14,14,4,11,15,3
Increase calcium uptake,16,16,0,0,0,0,14,10,1,15,...,2,12,8,1,12,8,3,11,8,4


In [52]:
# Create a DataFrame for unique peptide counts
peptide_absorbance_df= pd.DataFrame.from_dict(summed_function_absorbance, orient='index', columns=['Summed Abundance'])

# Create a DataFrame for unique function counts
function_absorbance_df= pd.DataFrame.from_dict(unique_function_absorbance, orient='index').fillna(0).astype(int)

# Concatenate the DataFrames
combined_absorbance_df = pd.concat([peptide_absorbance_df, function_absorbance_df], axis=1).T
combined_absorbance_df

,Control,New_Formula,Human_Milk,Human_Milk_Intestinal,Human_Milk_Gastric,Human_Milk_Feed,Control_Feed,Control_Gastric,Control_Intestinal,New_Formula_Feed,...,G_Intestinal,H_Feed,H_Gastric,H_Intestinal,I_Feed,I_Gastric,I_Intestinal,J_Feed,J_Gastric,J_Intestinal
Summed Abundance,2.685617e+10,2.754007e+10,2.006793e+10,2.979161e+10,1.328922e+10,7.651324e+08,4.540835e+08,7.334530e+09,5.564684e+10,6.991333e+09,...,6.228499e+10,3.989388e+08,7.479164e+09,5.942301e+10,5.141608e+08,8.828553e+09,5.167565e+10,5.005495e+08,1.277869e+10,5.106303e+10
ACE-inhibitory,3.354308e+09,3.608137e+09,1.128077e+08,1.450206e+08,2.701746e+07,2.200810e+05,1.756782e+07,6.937948e+08,7.062384e+09,3.447130e+08,...,1.109004e+10,1.430074e+07,6.386036e+08,1.061229e+10,1.422103e+07,8.815624e+08,6.480220e+09,2.933107e+07,1.455020e+09,8.196612e+09
Antianxiety,4.689068e+07,9.648805e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.080860e+05,2.937788e+06,1.007772e+08,5.576543e+07,...,1.934244e+08,3.399460e+05,2.549730e+06,1.405316e+08,3.033590e+05,3.271503e+06,1.207821e+08,9.031790e+05,5.087534e+06,1.530563e+08
Anticancer,1.352360e+08,1.862417e+08,8.587552e+07,1.487163e+08,4.212300e+04,7.478000e+03,2.208105e+06,9.009221e+07,8.216490e+07,4.613393e+07,...,8.773736e+07,3.121075e+06,1.236344e+08,7.633722e+07,2.483975e+06,7.851078e+07,6.145014e+07,1.245571e+07,2.723134e+08,6.389341e+07
Antihypertensive,1.434303e+06,2.179271e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.434303e+06,0.000000e+00,5.152286e+07,...,0.000000e+00,0.000000e+00,7.552630e+05,0.000000e+00,0.000000e+00,2.204552e+06,0.000000e+00,0.000000e+00,2.343725e+06,4.369800e+04
Antimicrobial,1.650255e+09,2.082891e+09,3.802231e+09,8.102326e+07,9.689494e+09,2.428194e+07,2.671363e+07,8.785772e+08,2.802332e+09,1.107534e+09,...,2.831347e+09,2.809058e+07,6.986603e+08,2.405670e+09,2.114373e+07,1.117703e+09,2.117419e+09,3.244987e+07,1.444462e+09,1.640999e+09
Antioxidant,3.237278e+09,3.545112e+09,1.785175e+08,4.462391e+08,5.064600e+04,2.932800e+04,1.212601e+07,2.739810e+08,8.527323e+09,8.689136e+08,...,1.245899e+10,9.741316e+06,2.570919e+08,1.161055e+10,8.042312e+06,3.633769e+08,7.430087e+09,1.476998e+07,5.183453e+08,8.445203e+09
Antithrombitic,5.365110e+05,6.154810e+05,6.129710e+05,0.000000e+00,6.129710e+05,0.000000e+00,1.810220e+05,7.297300e+05,4.617900e+05,4.862920e+05,...,7.064390e+05,0.000000e+00,3.693130e+05,4.110730e+05,1.507130e+05,9.434660e+05,6.871280e+05,0.000000e+00,8.785810e+05,2.952820e+05
Antithrombotic,6.923778e+07,1.308518e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.732950e+06,1.057207e+08,6.740015e+06,7.152539e+06,...,7.159770e+06,3.248826e+06,1.439297e+08,7.659591e+06,2.751685e+06,9.427565e+07,3.985199e+06,1.288049e+07,3.034712e+08,6.272896e+06
Bradykinin-Potentiating,1.033144e+06,1.831482e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.033144e+06,0.000000e+00,...,1.952034e+06,0.000000e+00,0.000000e+00,1.524373e+06,0.000000e+00,0.000000e+00,1.240571e+06,0.000000e+00,0.000000e+00,1.996188e+06


In [61]:
def format_abundance_counts(abundance_df, count_df):
    # Create an empty DataFrame with the same shape as the input DataFrames
    formatted_df = pd.DataFrame(index=abundance_df.index, columns=abundance_df.columns)
    
    # Format each cell
    for col in abundance_df.columns:
        for idx in abundance_df.index:
            # Get abundance value
            abundance = abundance_df.loc[idx, col]
            
            # Handle the special case for first row
            if idx == 'Summed Abundance':
                count = count_df.loc['Counts of peptides', col]
            else:
                # For all other rows, use the same index
                count = count_df.loc[idx, col]
            
            # Check if both abundance and count are zero
            if abundance == 0 and count == 0:
                formatted_value = "-"
            else:
                # Format abundance in scientific notation and count to rounded integer
                formatted_value = f"{abundance:.2e} ({round(count)})"
            
            # Store in new DataFrame
            formatted_df.loc[idx, col] = formatted_value
    
    # Rename the first row from 'Summed Abundance' to 'Total'
    formatted_df.rename(index={'Summed Abundance': 'Total'}, inplace=True)
    
    return formatted_df

# Create the formatted DataFrame and save to CSV
result_df = format_abundance_counts(combined_absorbance_df, combined_count_df)
result_df.to_csv('test.csv')